# introduction 

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import sys 
from sklearn.impute import SimpleImputer
import random
from plotly.offline import iplot
from plotnine import ggplot, aes, geom_line, geom_bar, geom_point, labs, coord_flip, theme_xkcd, geom_smooth


# Mauna Loa Data

In [28]:
mauna_loa_data = pd.read_csv('data/mlo_data.csv',
                             skiprows = [i for i in range(1, 44)])

In [33]:
mauna_loa_data.head()# checking the head of the dataset

,year,mean,unc
0,2002,373.45,0.12
1,2003,375.98,0.12
2,2004,377.70,0.12
3,2005,379.98,0.12
4,2006,382.09,0.12


In [38]:
mauna_loa_data.tail() 

,year,mean,unc
16,2018,408.72,0.12
17,2019,411.65,0.12
18,2020,414.21,0.12
19,2021,416.41,0.12
20,2022,418.53,0.12


In [44]:
mauna_loa_data.info() #getting the info on dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    21 non-null     int64  
 1   mean    21 non-null     float64
 2   unc     21 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 636.0 bytes


# polar ice

In [45]:
seaice = pd.read_csv('data/seaice.csv') #reading in the seaice dataset


In [46]:
seaice.head() ##checking the start of the dataset to see the variable and headings

,Year,Month,Day,Extent,Missing,Source Data,hemisphere
0,1978,10,26,10.231,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
1,1978,10,28,10.420,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
2,1978,10,30,10.557,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
3,1978,11,1,10.670,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north
4,1978,11,3,10.777,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsid...,north


In [47]:
seaice.tail() #checking the end of this dataset again to see the variables 

,Year,Month,Day,Extent,Missing,Source Data,hemisphere
26349,2019,5,27,10.085,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsidc...,south
26350,2019,5,28,10.078,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsidc...,south
26351,2019,5,29,10.219,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsidc...,south
26352,2019,5,30,10.363,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsidc...,south
26353,2019,5,31,10.436,0.0,['ftp://sidads.colorado.edu/pub/DATASETS/nsidc...,south


In [49]:
seaice.info() #checking the info on dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26354 entries, 0 to 26353
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          26354 non-null  int64  
 1    Month        26354 non-null  int64  
 2    Day          26354 non-null  int64  
 3        Extent   26354 non-null  float64
 4       Missing   26354 non-null  float64
 5    Source Data  26354 non-null  object 
 6   hemisphere    26354 non-null  object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.4+ MB


In [50]:
southen = seaice[seaice.iloc[:,6]=='south'] #taking only the information from the south hempisphere

In [51]:
years_list = list(southen['Year'].value_counts().index)
years_list = sorted(years_list)